In [1]:
import os

import tensorflow as tf

import pickle
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D

from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

import warnings

import cv2

In [2]:
os.listdir('./weights/ResNet50')

['AveragePooling2D&Adam', 'MaxPooling2D&Adam']

In [3]:
path1 = "./weights/ResNet50/AveragePooling2D&Adam/"

path2 = "./weights/ResNet50/MaxPooling2D&Adam/"

In [5]:
from tensorflow.keras.applications import ResNet50

# Average Pooling 2D

### Loading weights and model architecture

In [6]:
baseModel = ResNet50(weights="imagenet", include_top=False, input_tensor=Input(shape=(240, 320, 3)))
# baseModel = Xception(weights="imagenet", include_top=False, input_tensor=Input(shape=(240, 320, 3)))

headModel = baseModel.output

headModel = AveragePooling2D(pool_size=(7, 7),)(headModel)

headModel = Flatten(name="flatten")(headModel)

headModel = Dense(512, kernel_regularizer=l2(0.0001), activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)

headModel = Dense(1, activation="sigmoid")(headModel)

model = Model(inputs=baseModel.input, outputs=headModel)

for layer in baseModel.layers:
    layer.trainable = False
    
model.load_weights(path1+'model_weights.h5')

In [7]:
os.listdir('./dataset/VideoDataset/test')[:20]

['Burglary001_x264.mp4',
 'Burglary002_x264.mp4',
 'Burglary003_x264.mp4',
 'Burglary013_x264.mp4',
 'Burglary014_x264.mp4',
 'Burglary015_x264.mp4',
 'Burglary025_x264.mp4',
 'Burglary026_x264.mp4',
 'Burglary027_x264.mp4',
 'Burglary085_x264.mp4',
 'Normal_Videos_006_x264.mp4',
 'Normal_Videos_015_x264.mp4',
 'Normal_Videos_018_x264.mp4',
 'Normal_Videos_150_x264.mp4',
 'Normal_Videos_168_x264.mp4',
 'Normal_Videos_189_x264.mp4',
 'Normal_Videos_251_x264.mp4',
 'Normal_Videos_310_x264.mp4',
 'Normal_Videos_439_x264.mp4',
 'Normal_Videos_453_x264.mp4']

### Model class:
#### 0 - Crime
#### 1 - Normal

We've inverted it like (1-predicted_value)*100 to get the probability of the video belonging to crime...

In [13]:
# Crime:
# group of  people enter a store and steal stuff: Burglary015_x264 ... initially looks normal, but model predict it to be
#  crime from the beginning.

# example where model is confused - guy enters a room and opens the door of something (maybe a fridge?): Burglary002_x264
# we need to make to model to predict only when it detects movement

# atm robbery: Burglary013_x264 ... same as above ie motion detection needed

# thieves in a car crash into some shop: Burglary027_x264 ... same as above ie motion detection needed




# Normal:
# not a crime: Normal_Videos_168_x264

# game of basketball: Normal_Videos_310_x264

# a pathway in an airport: Normal_Videos_150_x264

# a busy road: Normal_Videos_006_x264


Q = []

thresh = None
videoName = input("Enter a video present in test folder: ")

cap = cv2.VideoCapture(f'./dataset/VideoDataset/test/{videoName}.mp4')

if cap.isOpened()==False:
    print("Either file not found or wrong codec used")

while cap.isOpened():
    ret, frame = cap.read()
    
    if ret==True:
        out = frame.copy()
        out = cv2.cvtColor(out, cv2.COLOR_BGR2RGB)
        out = cv2.resize(out, (320, 240)).astype("float32")
        
        pred = model.predict(np.expand_dims(out, axis=0))[0][0]
        
        state = f"Score: {(1 - pred)*100}% crime"
        frame = cv2.putText(frame, state, (10,30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), 2)
        
        # feedback
        cv2.imshow('frame',frame)
    
        # if you want to quit, press 'q'
        if cv2.waitKey(1) & 0xFF==ord('q'):
            break
    
    # if nothing is returned, break out of the loop
    else:
        break

cap.release()
cv2.destroyAllWindows()

Enter a video present in test folder: Burglary013_x264


# Max Pooling 2D

### Loading weights and model architecture

In [16]:
baseModel = ResNet50(weights="imagenet", include_top=False, input_tensor=Input(shape=(240, 320, 3)))
# baseModel = Xception(weights="imagenet", include_top=False, input_tensor=Input(shape=(240, 320, 3)))

headModel = baseModel.output

headModel = MaxPooling2D(pool_size=(7, 7),)(headModel)

headModel = Flatten(name="flatten")(headModel)

headModel = Dense(512, kernel_regularizer=l2(0.0001), activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)

headModel = Dense(1, activation="sigmoid")(headModel)

model = Model(inputs=baseModel.input, outputs=headModel)

for layer in baseModel.layers:
    layer.trainable = False
    
model.load_weights(path2+'model_weights.h5')

In [17]:
os.listdir('./dataset/VideoDataset/test')[:20]

['Burglary001_x264.mp4',
 'Burglary002_x264.mp4',
 'Burglary003_x264.mp4',
 'Burglary013_x264.mp4',
 'Burglary014_x264.mp4',
 'Burglary015_x264.mp4',
 'Burglary025_x264.mp4',
 'Burglary026_x264.mp4',
 'Burglary027_x264.mp4',
 'Burglary085_x264.mp4',
 'Normal_Videos_006_x264.mp4',
 'Normal_Videos_015_x264.mp4',
 'Normal_Videos_018_x264.mp4',
 'Normal_Videos_150_x264.mp4',
 'Normal_Videos_168_x264.mp4',
 'Normal_Videos_189_x264.mp4',
 'Normal_Videos_251_x264.mp4',
 'Normal_Videos_310_x264.mp4',
 'Normal_Videos_439_x264.mp4',
 'Normal_Videos_453_x264.mp4']

### Model class:
#### 0 - Crime
#### 1 - Normal

We've inverted it like (1-predicted_value)*100 to get the probability of the video belonging to crime...

In [25]:
# Crime:
# group of  people enter a store and steal stuff: Burglary015_x264 ... initially looks normal, but model predict it to be
#  crime from the beginning.

# example where model is confused - guy enters a room and opens the door of something (maybe a fridge?): Burglary002_x264
# we need to make to model to predict only when it detects movement

# atm robbery: Burglary013_x264 ... same as above ie motion detection needed

# thieves in a car crash into some shop: Burglary027_x264 ... same as above ie motion detection needed




# Normal:
# not a crime: Normal_Videos_168_x264

# game of basketball: Normal_Videos_310_x264

# a pathway in an airport: Normal_Videos_150_x264 ... unlike the previous model, here we can observe some confusion predicting
# wether it is normal or not

# a busy road: Normal_Videos_006_x264


Q = []

thresh = None
videoName = input("Enter a video present in test folder: ")

cap = cv2.VideoCapture(f'./dataset/VideoDataset/test/{videoName}.mp4')

if cap.isOpened()==False:
    print("Either file not found or wrong codec used")

while cap.isOpened():
    ret, frame = cap.read()
    
    if ret==True:
        out = frame.copy()
        out = cv2.cvtColor(out, cv2.COLOR_BGR2RGB)
        out = cv2.resize(out, (320, 240)).astype("float32")
        
        pred = model.predict(np.expand_dims(out, axis=0))[0][0]
        
        state = f"Score: {(1 - pred)*100}% crime"
        frame = cv2.putText(frame, state, (10,30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), 2)
        
        # feedback
        cv2.imshow('frame',frame)
    
        # if you want to quit, press 'q'
        if cv2.waitKey(1) & 0xFF==ord('q'):
            break
    
    # if nothing is returned, break out of the loop
    else:
        break

cap.release()
cv2.destroyAllWindows()

Enter a video present in test folder: Normal_Videos_006_x264
